In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"
year = 2019

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
display(running_order.shape)
running_order.head()

(5897, 20)

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,1,Stora Tuna OK 1,Stora Tuna OK,SWE,1,6.0,tilda östberg,1,Tilda Östberg,1.0,2.052,0.147,6.511986,1.881337,1.978520,0.048591,0.100000,2.052,0.100000
1,1,1,Stora Tuna OK 1,Stora Tuna OK,SWE,2,5.7,magdalena olsson,2,Magdalena Olsson,6.0,1.896,0.108,6.992522,1.765439,1.961637,0.098099,0.100000,1.896,0.108000
2,2,1,Stora Tuna OK 1,Stora Tuna OK,SWE,3,7.3,julia jakob,3,Julia Jakob,1.0,1.922,0.138,6.569075,1.798748,2.018901,0.110076,0.110076,1.922,0.110076
3,3,1,Stora Tuna OK 1,Stora Tuna OK,SWE,4,7.9,tove alexandersson,4,Tove Alexandersson,6.0,1.867,0.111,6.290935,1.791712,1.973971,0.091129,0.100000,1.867,0.111000
4,4,3,MS Parma 1,MS Parma,FIN,1,6.0,anastasia rudnaya,1,Anastasia Rudnaya,4.0,1.928,0.096,6.891132,1.875490,2.002510,0.063510,0.100000,1.928,0.096000


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std
0,1,Stora Tuna OK 1,1,6.0,Tilda Östberg,2.052,0.100000
1,1,Stora Tuna OK 1,2,5.7,Magdalena Olsson,1.896,0.108000
2,1,Stora Tuna OK 1,3,7.3,Julia Jakob,1.922,0.110076
3,1,Stora Tuna OK 1,4,7.9,Tove Alexandersson,1.867,0.111000
4,3,MS Parma 1,1,6.0,Anastasia Rudnaya,1.928,0.096000


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                            \
leg                        1                    2                    3   
team_id                                                                  
1            Stora Tuna OK 1      Stora Tuna OK 1      Stora Tuna OK 1   
3                 MS Parma 1           MS Parma 1           MS Parma 1   
4        Järla Orientering 1  Järla Orientering 1  Järla Orientering 1   
5             Alfta ÖSA OK 1       Alfta ÖSA OK 1       Alfta ÖSA OK 1   
6                Ol Norska 1          Ol Norska 1          Ol Norska 1   

                             dist                               name  \
leg                        4    1    2    3    4                   1   
team_id                                                                
1            Stora Tuna OK 1  6.0  5.7  7.3  7.9       Tilda Östberg   
3                 MS Parma 1  6.0  5.7  7.3  7.9   Anastasia Rudnaya   
4        Järla Orientering 1  6.0  5.7  7.3  7.9       Sara Sjökvist   
5             Alfta ÖSA OK 1  6.0  5.7  7.3  7.9  Galina Vinogradova   
6                Ol Norska 1  6.0  5.7  7.3  7.9         Anja Probst   

                                                                       \
leg                        2                    3                   4   
team_id                                                                 
1           Magdalena Olsson          Julia Jakob  Tove Alexandersson   
3             Anni Heikkonen       Tiia Istolahti       Maija Sianoja   
4        Anna Dahlgren-Rosén  Elin Hemmyr-Skantze      Sofie Bachmann   
5            Josefine Heikka         Irina Nyberg    Natalia Gemperle   
6          Valérie Aebischer         Siri Nyfeler       Sarina Jenzer   

        log_mean                         log_std                             
leg            1         2      3      4       1         2         3      4  
team_id                                                                      
1          2.052  1.896000  1.922  1.867   0.100  0.108000  0.110076  0.111  
3          1.928  2.046000  1.916  1.950   0.096  0.098000  0.128204  0.086  
4          1.916  1.938729  1.903  2.033   0.116  0.108902  0.124320  0.100  
5          1.904  1.925000  1.895  1.905   0.090  0.117383  0.092000  0.099  
6          2.232  1.925998  2.187  1.944   0.100  0.107078  0.153484  0.092

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(1482, 20)

(1460, 20)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
changeover_closing_timedelta = shared.changeover_closing[ve_or_ju][year] - shared.start_timestamp[ve_or_ju][year]
changeover_closing_mins = changeover_closing_timedelta.total_seconds() / 60
display(changeover_closing_mins)

270.0

In [8]:
if ve_or_ju == "ju":
    dark_period_start_timedelta = shared.dark_period[year]["start"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_start_mins = dark_period_start_timedelta.total_seconds() / 60
    dark_period_end_timedelta = shared.dark_period[year]["end"] - shared.start_timestamp[ve_or_ju][year]
    dark_period_end_mins = dark_period_end_timedelta.total_seconds() / 60
else:
    # hack to disbale calculation for Venlas
    dark_period_start_mins = 10000
    dark_period_end_mins = -1
    

In [9]:

# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    personal_start95 = samples.quantile(0.025)
    personal_end95 = samples.quantile(0.975)
    personal_median = samples.median()
        
    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    before_changeover_closing_samples = samples_sums < changeover_closing_mins
    
    mass_start = 1 - before_changeover_closing_samples.mean()
    # mass start of runner depends on arrival of previous runner
    mass_start = mass_start.shift(1, fill_value=0)

    not_dark_during_leg_samples = pd.DataFrame()
    for i in range(1,num_legs + 1):
        if i == 1:
            start = 0
        else:    
            start = samples_sums[i - 1]
            
        finish = samples_sums[i]
        finish_before_sunset = finish < dark_period_start_mins
        start_after_sunrise = start > dark_period_end_mins
        not_dark_during_leg_samples[i] = finish_before_sunset | start_after_sunrise 
    
    dark_during_leg = 1 - not_dark_during_leg_samples.mean()    
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    mass_start_dict = {f"mass_start_{leg}" : mass_start.values[leg-1] for leg in range(1, num_legs + 1)}
    dark_during_leg_dict = {f"dark_during_leg_{leg}" : dark_during_leg.values[leg-1] for leg in range(1, num_legs + 1)}

    personal_start95_dict = {}
    personal_end95_dict = {}
    personal_median_dict = {}
    for leg in range(1, num_legs + 1):
        personal_start95_dict[f"personal_start95_{leg}"] = personal_start95.values[leg-1]
        personal_end95_dict[f"personal_end95_{leg}"] = personal_end95.values[leg-1]
        personal_median_dict[f"personal_median_{leg}"] = personal_median.values[leg-1]
        
    new_cols = {**fin_start95_dict, 
                **fin_end95_dict, 
                **fin_median_dict, 
                **fin_mean_dict, 
                **fin_sum_log_means_dict, 
                **fin_sum_log_stds_dict, 
                **mass_start_dict, 
                **dark_during_leg_dict,
                **personal_start95_dict,
                **personal_end95_dict,
                **personal_median_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,38.440422,73.606437,119.560712,167.757512,56.972855,97.954704,152.206752,207.259102,46.829591,85.080808,...,40.059945,41.214651,56.972855,46.782907,61.787869,63.560205,46.829591,38.046577,49.810846,51.005204
3,34.272967,74.848798,119.399734,172.915042,49.798001,97.711934,153.852808,211.725288,41.285273,85.607573,...,38.751983,47.036712,49.798001,53.427493,63.809447,65.630906,41.285273,44.121119,49.594140,55.509412
4,32.468808,68.876772,113.631468,170.794563,51.371583,94.309610,148.592504,212.828121,40.758486,80.627201,...,38.232394,49.763966,51.371583,48.683917,62.546866,73.315591,40.758486,39.601038,48.837025,60.343813
5,33.706074,68.830631,114.235595,164.349707,48.006491,91.840090,143.507738,200.897778,40.232012,79.569782,...,40.301085,43.845020,48.006491,49.201775,58.146065,64.425053,40.232012,39.157857,48.571925,53.104142
6,45.906129,82.388747,138.965833,191.978069,68.013483,109.956860,188.075257,244.989597,55.930272,95.264456,...,48.228761,45.965307,68.013483,48.234534,87.876418,66.082721,55.930272,39.078900,65.212238,55.169403


In [10]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat.head().round(2)

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,name_3,name_4,log_mean_1,log_mean_2,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4
team_id,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,Julia Jakob,Tove Alexandersson,2.05,1.90,1.92,1.87,0.10,0.11,0.11,0.11
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,Tiia Istolahti,Maija Sianoja,1.93,2.05,1.92,1.95,0.10,0.10,0.13,0.09
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,Elin Hemmyr-Skantze,Sofie Bachmann,1.92,1.94,1.90,2.03,0.12,0.11,0.12,0.10
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,Irina Nyberg,Natalia Gemperle,1.90,1.92,1.90,1.90,0.09,0.12,0.09,0.10
6,Ol Norska 1,Ol Norska 1,Ol Norska 1,Ol Norska 1,6.0,5.7,7.3,7.9,Anja Probst,Valérie Aebischer,Siri Nyfeler,Sarina Jenzer,2.23,1.93,2.19,1.94,0.10,0.11,0.15,0.09


In [11]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [12]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,personal_start95_3,personal_start95_4,personal_end95_1,personal_end95_2,personal_end95_3,personal_end95_4,personal_median_1,personal_median_2,personal_median_3,personal_median_4
team_id,,,,,,,,,,,,,,,,,,,,,
1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,Stora Tuna OK 1,6.0,5.7,7.3,7.9,Tilda Östberg,Magdalena Olsson,...,40.059945,41.214651,56.972855,46.782907,61.787869,63.560205,46.829591,38.046577,49.810846,51.005204
3,MS Parma 1,MS Parma 1,MS Parma 1,MS Parma 1,6.0,5.7,7.3,7.9,Anastasia Rudnaya,Anni Heikkonen,...,38.751983,47.036712,49.798001,53.427493,63.809447,65.630906,41.285273,44.121119,49.594140,55.509412
4,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,6.0,5.7,7.3,7.9,Sara Sjökvist,Anna Dahlgren-Rosén,...,38.232394,49.763966,51.371583,48.683917,62.546866,73.315591,40.758486,39.601038,48.837025,60.343813
5,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,Alfta ÖSA OK 1,6.0,5.7,7.3,7.9,Galina Vinogradova,Josefine Heikka,...,40.301085,43.845020,48.006491,49.201775,58.146065,64.425053,40.232012,39.157857,48.571925,53.104142
6,Ol Norska 1,Ol Norska 1,Ol Norska 1,Ol Norska 1,6.0,5.7,7.3,7.9,Anja Probst,Valérie Aebischer,...,48.228761,45.965307,68.013483,48.234534,87.876418,66.082721,55.930272,39.078900,65.212238,55.169403


In [13]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist                name  log_mean   log_std  \
team_id leg                                                                  
1       1    Stora Tuna OK 1   6.0       Tilda Östberg     2.052  0.100000   
        2    Stora Tuna OK 1   5.7    Magdalena Olsson     1.896  0.108000   
        3    Stora Tuna OK 1   7.3         Julia Jakob     1.922  0.110076   
        4    Stora Tuna OK 1   7.9  Tove Alexandersson     1.867  0.111000   
3       1         MS Parma 1   6.0   Anastasia Rudnaya     1.928  0.096000   
        2         MS Parma 1   5.7      Anni Heikkonen     2.046  0.098000   
        3         MS Parma 1   7.3      Tiia Istolahti     1.916  0.128204   

             fin_start95   fin_end95  fin_median    fin_mean  \
team_id leg                                                    
1       1      38.440422   56.972855   46.829591   47.005001   
        2      73.606437   97.954704   85.080808   85.232005   
        3     119.560712  152.206752  135.136325  135.358035   
        4     167.757512  207.259102  186.520482  186.753768   
3       1      34.272967   49.798001   41.285273   41.462587   
        2      74.848798   97.711934   85.607573   85.771849   
        3     119.399734  153.852808  135.438375  135.813704   

             fin_sum_log_mean  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg                                                                   
1       1            3.845274         0.099818         0.0              0.0   
        2            4.442727         0.072826         0.0              0.0   
        3            4.906029         0.061566         0.0              0.0   
        4            5.228341         0.053842         0.0              0.0   
3       1            3.720197         0.095857         0.0              0.0   
        2            4.449348         0.068463         0.0              0.0   
        3            4.909202         0.064510         0.0              0.0   

             personal_start95  personal_end95  personal_median  
team_id leg                                                     
1       1           38.440422       56.972855        46.829591  
        2           30.735077       46.782907        38.046577  
        3           40.059945       61.787869        49.810846  
        4           41.214651       63.560205        51.005204  
3       1           34.272967       49.798001        41.285273  
        2           36.360375       53.427493        44.121119  
        3           38.751983       63.809447        49.594140

In [14]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95", "mass_start", "dark_during_leg"]]

,,name,fin_start95,fin_median,fin_end95,mass_start,dark_during_leg
team_id,leg,,,,,,


In [15]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= start_timestamp)

runner_estimates["personal_estimate_interval"] = runner_estimates["personal_end95"] - runner_estimates["personal_start95"]
runner_estimates.head()

team  dist                name  log_mean   log_std  \
team_id leg                                                                  
1       1    Stora Tuna OK 1   6.0       Tilda Östberg     2.052  0.100000   
        2    Stora Tuna OK 1   5.7    Magdalena Olsson     1.896  0.108000   
        3    Stora Tuna OK 1   7.3         Julia Jakob     1.922  0.110076   
        4    Stora Tuna OK 1   7.9  Tove Alexandersson     1.867  0.111000   
3       1         MS Parma 1   6.0   Anastasia Rudnaya     1.928  0.096000   

             fin_start95   fin_end95  fin_median    fin_mean  \
team_id leg                                                    
1       1      38.440422   56.972855   46.829591   47.005001   
        2      73.606437   97.954704   85.080808   85.232005   
        3     119.560712  152.206752  135.136325  135.358035   
        4     167.757512  207.259102  186.520482  186.753768   
3       1      34.272967   49.798001   41.285273   41.462587   

             fin_sum_log_mean  fin_sum_log_std  mass_start  dark_during_leg  \
team_id leg                                                                   
1       1            3.845274         0.099818         0.0              0.0   
        2            4.442727         0.072826         0.0              0.0   
        3            4.906029         0.061566         0.0              0.0   
        4            5.228341         0.053842         0.0              0.0   
3       1            3.720197         0.095857         0.0              0.0   

             personal_start95  personal_end95  personal_median  \
team_id leg                                                      
1       1           38.440422       56.972855        46.829591   
        2           30.735077       46.782907        38.046577   
        3           40.059945       61.787869        49.810846   
        4           41.214651       63.560205        51.005204   
3       1           34.272967       49.798001        41.285273   

                          fin_time_median              fin_time_start95  \
team_id leg                                                               
1       1   2019-06-15 14:46:49.775468826 2019-06-15 14:38:26.425346136   
        2   2019-06-15 15:25:04.848502159 2019-06-15 15:13:36.386239290   
        3   2019-06-15 16:15:08.179514408 2019-06-15 15:59:33.642746687   
        4   2019-06-15 17:06:31.228931665 2019-06-15 16:47:45.450699568   
3       1   2019-06-15 14:41:17.116374254 2019-06-15 14:34:16.378008366   

                           fin_time_end95  personal_estimate_interval  
team_id leg                                                            
1       1   2019-06-15 14:56:58.371294260                   18.532432  
        2   2019-06-15 15:37:57.282251835                   16.047830  
        3   2019-06-15 16:32:12.405102015                   21.727924  
        4   2019-06-15 17:27:15.546137333                   22.345554  
3       1   2019-06-15 14:49:47.880081892                   15.525035

In [16]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")

#for_print[for_print['team'].str.contains("Stadin")]
for_print = for_print[for_print['team'].str.contains("Reak")]

for_print = for_print[[
 'leg',
 'name', 
 'personal_start95', 
 'personal_median',
 'personal_end95',
 'personal_estimate_interval']]

for_print.to_csv(f'data/for_print_{ve_or_ju}_2019.csv')
for_print

,leg,name,personal_start95,personal_median,personal_end95,personal_estimate_interval
team_id,,,,,,


In [17]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'ind_log_mean', 
 'ind_log_std',
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median',
 'mass_start', 
 'dark_during_leg']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Reak")]

,team_id,leg,team,name,ind_log_mean,ind_log_std,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median,mass_start,dark_during_leg
1016,341,1,Rasti-Jyry 1,Jola Maisala,2.197000,0.117000,3.989501,0.117829,2019-06-15 11:42:47.125176190+00:00,2019-06-15 12:08:08.595057964+00:00,2019-06-15 11:54:02.240598679+00:00,0.0000,0.0
1017,341,2,Rasti-Jyry 1,Kaija Rantoja,2.364000,0.140000,4.745685,0.093008,2019-06-15 12:36:08.988414764+00:00,2019-06-15 13:18:16.823383808+00:00,2019-06-15 12:55:06.947766542+00:00,0.0000,0.0
1018,341,3,Rasti-Jyry 1,Jenni Castren,2.395000,0.152000,5.277417,0.083913,2019-06-15 13:46:22.912232161+00:00,2019-06-15 14:50:49.163397074+00:00,2019-06-15 14:15:49.656412840+00:00,0.0000,0.0
1019,341,4,Rasti-Jyry 1,Inka Lemmetyinen,2.558000,0.178309,5.700941,0.083264,2019-06-15 15:15:04.373195410+00:00,2019-06-15 16:53:14.661331892+00:00,2019-06-15 15:58:53.540324688+00:00,0.0000,0.0
1696,528,1,Rasti-Jyry 2,Päivi Taponen,2.418000,0.163000,4.211646,0.165343,2019-06-15 11:49:06.649562597+00:00,2019-06-15 12:33:36.333875656+00:00,2019-06-15 12:07:23.608102560+00:00,0.0000,0.0
1697,528,2,Rasti-Jyry 2,Miia Hovi,2.229000,0.141000,4.796795,0.111356,2019-06-15 12:37:28.591860294+00:00,2019-06-15 13:31:02.753097296+00:00,2019-06-15 13:01:03.525359392+00:00,0.0000,0.0
1698,528,3,Rasti-Jyry 2,Mervi Luurila,2.459000,0.191317,5.335187,0.102340,2019-06-15 13:50:06.794873476+00:00,2019-06-15 15:14:19.029824972+00:00,2019-06-15 14:27:18.247355223+00:00,0.0000,0.0
1699,528,4,Rasti-Jyry 2,Tiina Isoniemi,2.371000,0.116000,5.679978,0.080081,2019-06-15 15:10:25.337016344+00:00,2019-06-15 16:42:39.408994675+00:00,2019-06-15 15:52:41.163505316+00:00,0.0056,0.0
2448,740,1,Rasti-Jyry 4,Mila Viksilä,2.581289,0.144432,4.374548,0.144609,2019-06-15 11:59:30.671234369+00:00,2019-06-15 12:45:18.903222322+00:00,2019-06-15 12:19:33.579717398+00:00,0.0000,0.0
2449,740,2,Rasti-Jyry 4,Tuula Ohramaa,2.617000,0.174000,5.067166,0.113796,2019-06-15 13:07:17.628474236+00:00,2019-06-15 14:18:59.314754486+00:00,2019-06-15 13:38:24.189467192+00:00,0.0000,0.0


In [18]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")

In [19]:
for_web.shape

(5840, 13)